In [ ]:
### Import tensorflow, keras, and numpy
import tensorflow as tf
from tensorflow import keras
import numpy as np

2021-08-06 09:37:02.911347: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-06 09:37:02.911401: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
### Import required objects from keras
from keras.datasets import mnist
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPool2D
from keras.layers import Activation, Input, Flatten, Dropout
from keras.models import Model
from keras.utils import to_categorical

In [ ]:
### Load in the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

### Number of classes
print(f'The dataset has {len(np.unique(y_train))} distinct classes.')

### Scale down the datasets
X_train, X_test = X_train.astype('float32')/255, X_test.astype('float32')/255

### Network params
IMAGE_SIZE = X_train.shape[1]
BATCH_SIZE = 64
INPUT_SHAPE = [IMAGE_SIZE, IMAGE_SIZE, 1] ### Image width, image height, number of channels

11493376/11490434 [==============================] - 0s 0us/step
The dataset has 10 distinct classes.


In [ ]:
IMAGE_SIZE

28

In [ ]:
INPUT_SHAPE

[28, 28, 1]

In [ ]:
X_train.shape

(60000, 28, 28)

In [ ]:
y_train.shape

(60000,)

In [ ]:
y_test.shape

(10000,)

In [ ]:
### Reshape and one-hot encode class labels
y_train = np.reshape(y_train, newshape = (60000, 1))
y_test = np.reshape(y_test, newshape = (10000, 1))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
### Build Encoder
def build_encoder():
    inputs = Input(shape = INPUT_SHAPE, name = 'Input layer')
    ### 28 BY 28
    x = Conv2D(filters = 6, kernel_size = 3, padding = 'same') (inputs)
    x = MaxPool2D(pool_size = 2, padding = 'same') (x)
    ### 26 BY 26
    x = Conv2D(filters = 16, kernel_size = 3, padding = 'same', activation = 'relu') (x)
    x = MaxPool2D(pool_size = 2, padding = 'same') (x)
    ### 24 BY 24
    x = Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu') (x)
    x = MaxPool2D(pool_size = 2, padding = 'same') (x)
    ### 22 BY 22
    x = Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu') (x)
    x = MaxPool2D(pool_size = 2, padding = 'same') (x)

    ### Flatten
    x = Flatten() (x)
    x = Dropout(0.2) (x)
    x = Dense(128, activation = 'relu') (x)
    x = Dropout(0.2) (x)
    x = Dense(64, activation = 'relu') (x)
    x = Dropout(0.2) (x)
    x = Dense(10, activation = 'softmax') (x)

    return Model(inputs = inputs, outputs = x)

In [ ]:
encoder = build_encoder()

2021-08-06 09:37:11.658284: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-06 09:37:11.662342: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-06 09:37:11.662368: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-06 09:37:11.662391: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-cedde139-93f2-4278-8e9d-d80931c4b7dc): /proc/driver/nvidia/version does not exist
2021-08-06 09:37:11.662619: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operatio

In [ ]:
### Encoder summary
encoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input layer (InputLayer)     [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 6)         60        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 16)        880       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0     

In [ ]:
### Import and instantiate optimizer
from keras.optimizers import RMSprop, SGD
optimizer = SGD(learning_rate = 0.001, momentum = 0.0)

### Compile model
encoder.compile(loss = 'categorical_crossentropy', optimizer = optimizer,
                metrics = ['accuracy'])

In [ ]:
### Import and instantiate a learning rate scheduler
from keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(monitor = 'val_loss', patience = 5,
                                 verbose = 1, min_lr = 0.0001, factor = 0.1)

In [ ]:
### Fit model
encoder.fit(X_train, y_train, epochs = 100, batch_size = BATCH_SIZE, verbose = 2,
            callbacks = [lr_scheduler], validation_data = (X_test, y_test))

2021-08-06 09:37:11.983137: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 188160000 exceeds 10% of free system memory.
2021-08-06 09:37:12.260078: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-06 09:37:12.271308: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2500000000 Hz
Epoch 1/100
2021-08-06 09:37:54.336445: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 31360000 exceeds 10% of free system memory.
938/938 - 45s - loss: 2.2985 - accuracy: 0.1180 - val_loss: 2.2862 - val_accuracy: 0.2021
Epoch 2/100
938/938 - 40s - loss: 2.2848 - accuracy: 0.1564 - val_loss: 2.2681 - val_accuracy: 0.4089
Epoch 3/100
938/938 - 40s - loss: 2.2649 - accuracy: 0.1983 - val_loss: 2.2343 - val_accuracy: 0.4533
Epoch 4/100
938/938 - 39s - loss: 2.2274 - accuracy: 0.2420 - val_loss: 2.1580 - val_accuracy: 0.5277
Epoch 5/100
938/938 - 39s - loss: 2.139

In [ ]:
### Check model performance
train_loss, train_acc = encoder.evaluate(X_train, y_train, verbose = 1)
test_loss, test_acc = encoder.evaluate(X_test, y_test, verbose = 1)

313/313 [==============================] - 2s 7ms/step - loss: 0.0478 - accuracy: 0.9848


In [ ]:
### Loss on train dataset
train_loss

0.04214317351579666

In [ ]:
### Accuracy on train dataset
train_acc

0.9869499802589417

In [ ]:
### Loss on test dataset
test_loss

0.04783645644783974

In [ ]:
### Accuracy on test dataset
test_acc

0.9847999811172485

In [ ]:
### Save model weights in .h5 format
encoder.save_weights(filepath = '/work/MNIST_model_weights.h5', save_format = 'hdf5')

In [ ]:
### Save model weights in .hdf5 format
encoder.save_weights(filepath = '/work/MNIST_model_weights.hdf5', save_format = 'hdf5')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cedde139-93f2-4278-8e9d-d80931c4b7dc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>